In [1]:
import json 
import py2neo # library used to comunicate with database
from py2neo import Graph, Node, Relationship, Database, RelationshipMatcher


## Create object of the graph database along with its username and password

In [4]:
graph = Graph("http://localhost:7474", auth=("neo4j", "123456"))

# Convert Json file to graphs

In order to convert json source file to graph database, we use neo4j database which uses cypher query language. To visualize the graph, login to  http://localhost:7474

In [6]:
# load json file
with open('./data/rte.json') as data_file:
    json_ = json.load(data_file)

# write cypher query
query = '''
WITH {json} as document
UNWIND document.bundles AS p
MERGE (b: bundles {name:p.name, className:p.eClass}) SET b.symbolicName=p.symbolicName
FOREACH (imp IN p.imports | 
MERGE(im: PackagesImports {name:substring(imp.ref, 3,10)}) SET im.FullName=imp.ref 
MERGE(b)-[:Imports]->(im))

FOREACH (exp IN p.exports | 
    MERGE(ex: PackagesExports {name:substring(exp.ref, 3,10)} ) SET ex.FullName=exp.ref 
    MERGE(b)-[:Exports]->(ex))

FOREACH (pkg IN p.packages | 

    MERGE(pk: Packages {name:right(pkg.ref,3)} ) SET pk.FullName=pkg.ref 
    MERGE(b)-[:Uses_Pkgs]->(pk))
    
    // Iterate through components
    
FOREACH (comp IN p.components | 

    MERGE (b)-[:Uses_Components]-> (c: Components {name:comp.name} ) SET c.className=comp.eClass, 
        c.implementation = comp.implementation.ref,
        c.bundle = comp.bundle.ref
        
        // Iterate through provided services
        FOREACH (service in comp.providedServices| 
             MERGE(c)-[:uses_services]->(sr: ComponentServices {name: substring(service.ref, 3, size(service.ref)-1) }) )
        
    )

FOREACH (pkgfrag IN p.packageFragments |
    MERGE(b)-[:Pkg_fragment]->(fragment: PackageFragments {name:pkgfrag.eClass}) SET fragment.className = pkgfrag.eClass
    MERGE(fragment)-[:Pack_By_Frag]->(pack: FragPackages {name:pkgfrag.package.ref})
    MERGE(fragment)-[:Bundle_By_Frag]->(bund: FragBundle {name:substring(pkgfrag.bundle.ref, 3, size(pkgfrag.bundle.ref)-1)})

    // Since compilation units is list so we need to iterate
    FOREACH (cmp IN pkgfrag.compilationUnits | 
        MERGE(fragment)-[:compiled_By]->(u: Units{name:substring(cmp.name, 0, size(cmp.name)-5)}) SET u.className = cmp.eClass, u.Loc = cmp.LOC 

        // go through PkgFragment inside compilationUnits -> packageFragment
        MERGE(u)-[:compiledUnits_pkgFragment]->
        (CPkFrag: compiledUF {name:substring(cmp.packageFragment.ref, 3, size(cmp.packageFragment.ref)-2)}) 

        // go through PkgFragment inside compilationUnits -> topLevelType

        MERGE(u)-[:compiledUnits_topLevelType]->
        (CPtpLevelType: compiledTopLevelType {name:cmp.topLevelType.name}) SET  CPtpLevelType.className = cmp.topLevelType.eClass,
        CPtpLevelType.visibility = cmp.topLevelType.visibility, 
        CPtpLevelType.qualifiedName = cmp.topLevelType.qualifiedName

        // Inside topLevelType > compilationUnit

        MERGE(CPtpLevelType) -[:topLevelType_compilationUnit]->
        (topLevel_cUnit: compilationUnit {name: cmp.topLevelType.compilationUnit.ref } ) 
        SET topLevel_cUnit.ref = cmp.topLevelType.compilationUnit.ref


        // Inside topLevelType > references

        FOREACH (refer IN cmp.topLevelType.references |
            MERGE(CPtpLevelType)-[:topLevelType_reference]->(referTL: References {name:substring(refer.ref, 31, size(refer.ref)-1)})
            SET referTL.ref = refer.ref
        )
        
        // Inside topLevelType > External references

        FOREACH (erefer IN cmp.topLevelType.externalReferences |
            MERGE(CPtpLevelType)-[:topLevelType_EReference]->
                (ereferTL: eReferences {name:erefer.ref})
            SET ereferTL.ref = erefer.ref
        )
        
        // Inside topLevelType > Constructors

        FOREACH (const IN cmp.topLevelType.constructors |
            MERGE(CPtpLevelType)-[:topLevelType_Const]->
                (constTL: Constructor {name:const.eClass})
            SET constTL.eClass = const.eClass,
                constTL.visibility = const.visibility,
                constTL.LOC = const.LOC
        )
        
        // Inside topLevelType > methods

        FOREACH (methd IN cmp.topLevelType.methods |
            MERGE(CPtpLevelType)-[:Methods_Contains]->
                (methodTL: Methods {name:methd.name})
            SET methodTL.eClass = methd.eClass,
                methodTL.name = methd.name,
                methodTL.visibility = methd.visibility,
                methodTL.LOC = methd.LOC
        )
        
        // Inside topLevelType > fields

        FOREACH (field IN cmp.topLevelType.fields |
            MERGE(CPtpLevelType)-[:Methods_Contains]->
                (fieldTL: Methods {name:field.name})
            SET fieldTL.eClass = field.eClass,
                fieldTL.name = field.name,
                fieldTL.visibility = field.visibility,
                
                fieldTL.modify_0 = field.modifier[0],
                fieldTL.modify_1 = field.modifier[1]

                
                
        )
        
    )
)

MERGE (v: Version {name:("Version " + p.version.major)}) SET 
  v.className = p.version.eClass,
  v.major=p.version.major, 
  v.minor=p.version.minor,
  v.micro=p.version.micro,
  v.qualifier=p.version.qualifier
  
 MERGE(b)-[:VersionNum]->(v)  
 
// Iterate through services

WITH {json} as document
UNWIND document.services AS s

MERGE (ser: service {name:s.interfaceName, className:s.eClass}) SET ser.interface = s.interface.ref


// Iterate through externalTypes

WITH {json} as document
UNWIND document.externalTypes AS e
 
MERGE (eTyp: eTypes {name:e.qualifiedName, className:e.eClass}) 
'''


# To get relations of particular node
# MATCH (n:classes) where n.name='RCE Excel Component Execution' return (n)-[]->()

#send query to the database
graph.run(query, json=json_).data()


[]

# query from database

In [12]:
# to determine the packages that are imported by class "RCE Excel Component GUI Bundle" 
v = 'RCE Excel Component GUI Bundle'
nodes = graph.nodes.match("classes", name=v).first()

print ("Packages imported by class: " + v + " are: ")
list(graph.relationships.match((nodes, None), "Imports") 
              .limit(3))

Packages imported by class: RCE Excel Component GUI Bundle are: 


[(RCE Excel Component GUI Bundle)-[:Imports {}]->(bundles.47),
 (RCE Excel Component GUI Bundle)-[:Imports {}]->(bundles.48)]